In [3]:
#Room Class
class Room:
    def __init__(self, room_number, room_type, price_per_night, amenities):
        self.room_number = room_number          # int
        self.room_type = room_type              # str
        self.price_per_night = price_per_night  # float
        self.amenities = amenities              # List[str]
        self.availability_status = True         # bool (default: available)

    def mark_available(self):
        """Set room status to available."""
        self.availability_status = True

    def mark_unavailable(self):
        """Set room status to unavailable."""
        self.availability_status = False

In [4]:
#Guest Class
class Guest:
    def __init__(self, guest_id, name, contact_info):
        self.guest_id = guest_id                # int
        self.name = name                        # str
        self.contact_info = contact_info         # str
        self.loyalty_status = False             # bool (default: not enrolled)
        self.reservations = []                  # List[Reservation] (track bookings)

    def update_profile(self, new_name, new_contact):
        """Update guest details."""
        self.name = new_name
        self.contact_info = new_contact

    def view_reservations(self):
        """Return all reservations linked to this guest."""
        return self.reservations

In [5]:
#Reservation Class
class Reservation:
    def __init__(self, reservation_id, guest, room, check_in_date, check_out_date):
        self.reservation_id = reservation_id     # int
        self.guest = guest                       # Guest object (composition)
        self.room = room                         # Room object (composition)
        self.check_in_date = check_in_date       # date
        self.check_out_date = check_out_date     # date
        self.status = "Pending"                  # str (Pending/Confirmed/Cancelled)

    def confirm_booking(self):
        """Confirm reservation if room is available."""
        if self.room.availability_status:
            self.status = "Confirmed"
            self.room.mark_unavailable()
            self.guest.reservations.append(self)  # Link to guest's booking history
        else:
            raise ValueError("Room is already booked.")

    def cancel_booking(self):
        """Cancel reservation and free the room."""
        if self.status == "Confirmed":
            self.status = "Cancelled"
            self.room.mark_available()
            self.guest.reservations.remove(self)
        else:
            raise ValueError("Cannot cancel a non-confirmed booking.")

In [6]:
#Invoice Class
class Invoice:
    def __init__(self, invoice_id, reservation):
        self.invoice_id = invoice_id             # int
        self.reservation = reservation           # Reservation object (composition)
        self.amount = self.calculate_amount()    # float
        self.status = "Unpaid"                   # str (Unpaid/Paid)

    def calculate_amount(self):
        """Compute total cost based on reservation duration."""
        nights = (self.reservation.check_out_date - self.reservation.check_in_date).days
        return nights * self.reservation.room.price_per_night

    def generate_invoice(self):
        """Return invoice details as a formatted string."""
        return f"Invoice #{self.invoice_id}: ${self.amount} (Status: {self.status})"

    def process_payment(self):
        """Mark invoice as paid."""
        self.status = "Paid"

In [7]:
#Payment class
class Payment:
    def __init__(self, payment_id, invoice, payment_method):
        self.payment_id = payment_id             # int
        self.invoice = invoice                   # Invoice object (composition)
        self.payment_method = payment_method     # str (e.g., "Credit Card")
        self.payment_status = "Pending"          # str (Pending/Completed/Failed)

    def process_payment(self):
        """Process payment and update invoice status."""
        try:
            self.invoice.process_payment()       # Calls Invoice's method
            self.payment_status = "Completed"
        except Exception as e:
            self.payment_status = "Failed"
            raise Exception(f"Payment failed: {str(e)}")

In [8]:
#Loyalty Program
class LoyaltyProgram:
    def __init__(self, program_id, guest):
        self.program_id = program_id             # int
        self.guest = guest                       # Guest object (composition)
        self.points = 0                          # int (default: 0)
        self.rewards_available = ["Free Night", "Upgrade"]  # List[str]

    def add_points(self, points_earned):
        """Add points to the guest's loyalty account."""
        if self.guest.loyalty_status:
            self.points += points_earned
        else:
            raise ValueError("Guest is not enrolled in the loyalty program.")

    def redeem_rewards(self, reward):
        """Redeem a reward if points are sufficient."""
        if reward in self.rewards_available:
            if reward == "Free Night" and self.points >= 100:
                self.points -= 100
                return "Free Night reward claimed!"
            elif reward == "Upgrade" and self.points >= 50:
                self.points -= 50
                return "Room upgrade claimed!"
            else:
                raise ValueError("Not enough points for this reward.")
        else:
            raise ValueError("Invalid reward.")

In [9]:
#Service Request Class
class ServiceRequest:
    def __init__(self, request_id, guest, request_type):
        self.request_id = request_id             # int
        self.guest = guest                       # Guest object (composition)
        self.request_type = request_type         # str (e.g., "Room Cleaning")
        self.status = "Open"                     # str (Open/In Progress/Closed)

    def submit_request(self):
        """Submit a new service request."""
        if self.status == "Open":
            self.guest.service_requests.append(self)  # Link to guest's requests
        else:
            raise ValueError("Request already submitted.")

    def update_status(self, new_status):
        """Update the request's status."""
        valid_statuses = ["Open", "In Progress", "Closed"]
        if new_status in valid_statuses:
            self.status = new_status
        else:
            raise ValueError("Invalid status.")

In [11]:


# --- Workflow Execution (Part C) ---
print("=== Hotel Management System Simulation ===")

# 1. Initialize Objects
room_101 = Room(101, "Deluxe", 200.00, ["WiFi", "Mini-Cafe"])
guest_john = Guest(1, "Reem Alhamami", "Reem@Alhamami.com")
reservation = Reservation(1, guest_john, room_101, date(2025, 3, 15), date(2025, 4, 1))
invoice = Invoice(1, reservation)
payment = Payment(1, invoice, "Credit Card")
loyalty_program = LoyaltyProgram(1, guest_john)
guest_john.loyalty_status = True  # Activate loyalty program
service_request = ServiceRequest(1, guest_john, "Extra Towels")

# 2. Execute Workflow
print("\n=== Booking Phase ===")
reservation.confirm_booking()
print(f"Reservation Status: {reservation.status}")
print(f"Room 101 Available: {room_101.availability_status}")

print("\n=== Payment Phase ===")
print(invoice.generate_invoice())
payment.process_payment()
print(f"Payment Status: {payment.payment_status}")
print(f"Invoice Status: {invoice.status}")

print("\n=== Loyalty Program ===")
loyalty_program.add_points(150)
print(f"Loyalty Points: {loyalty_program.points}")
print(loyalty_program.redeem_rewards("Free Night"))

print("\n=== Service Request ===")
service_request.submit_request()
service_request.update_status("In Progress")
print(f"Service Status: {service_request.status}")

=== Hotel Management System Simulation ===

=== Booking Phase ===
Reservation Status: Confirmed
Room 101 Available: False

=== Payment Phase ===
Invoice #1: $3400.00 (Status: Unpaid)
Payment Status: Completed
Invoice Status: Paid

=== Loyalty Program ===
Loyalty Points: 150
Free Night reward claimed!

=== Service Request ===
Service Status: In Progress


In [12]:
#Testing using Error handling
from datetime import date, datetime

# --- Enhanced Error Classes ---
class BookingError(Exception):
    """Base class for booking-related errors"""
    pass

class PaymentError(Exception):
    """Base class for payment failures"""
    pass

class RoomNotAvailableError(BookingError):
    def __init__(self, room_number):
        super().__init__(f"Room {room_number} is already booked")

class InvalidDateError(BookingError):
    def __init__(self):
        super().__init__("Check-out date must be after check-in date")

class PaymentProcessingError(PaymentError):
    def __init__(self, message="Payment processing failed"):
        super().__init__(message)

# --- Updated Reservation Class with Error Handling ---
class Reservation:
    def __init__(self, reservation_id, guest, room, check_in_date, check_out_date):
        # Validate dates during initialization
        if check_in_date >= check_out_date:
            raise InvalidDateError()

        self.reservation_id = reservation_id
        self.guest = guest
        self.room = room
        self.check_in_date = check_in_date
        self.check_out_date = check_out_date
        self.status = "Pending"

    def confirm_booking(self):
        """Confirm reservation with comprehensive checks"""
        try:
            if not self.room.availability_status:
                raise RoomNotAvailableError(self.room.room_number)

            if self.status != "Pending":
                raise BookingError("Reservation already processed")

            self.status = "Confirmed"
            self.room.mark_unavailable()
            self.guest.reservations.append(self)

        except RoomNotAvailableError as e:
            print(f" Booking Failed: {e}")
            raise  # Re-raise for caller to handle
        except Exception as e:
            print(f" Unexpected error: {e}")
            raise BookingError("Booking confirmation failed")

# --- Updated Payment Class with Error Handling ---
class Payment:
    def process_payment(self):
        """Process payment with transaction safety"""
        try:
            if self.invoice.status == "Paid":
                raise PaymentError("Invoice already paid")

            # Simulate payment processing
            if self.payment_method.lower() not in ["credit card", "debit card"]:
                raise PaymentProcessingError("Unsupported payment method")

            # Simulate 10% chance of payment failure
            if datetime.now().second % 10 == 0:  # Random failure
                raise PaymentProcessingError("Bank declined transaction")

            self.invoice.process_payment()
            self.payment_status = "Completed"

        except PaymentError as e:
            self.payment_status = "Failed"
            print(f" Payment Failed: {e}")
            raise
        except Exception as e:
            self.payment_status = "Error"
            print(f" System Error: {e}")
            raise PaymentProcessingError()

# --- Testing the Error Handling ---
try:
    # Force error scenarios:
    room_101 = Room(101, "Deluxe", 200, ["WiFi"])
    guest = Guest(1, "Reem", "Reem@gmail.com")

    # 1. Test invalid dates
    try:
        bad_res = Reservation(1, guest, room_101, date(2025,3,20), date(2025,3,30))
    except InvalidDateError as e:
        print(f" Caught invalid dates: {e}")

    # 2. Test double booking
    res1 = Reservation(1, guest, room_101, date(2025,3,20), date(2025,3,30))
    res1.confirm_booking()

    try:
        res2 = Reservation(2, guest, room_101, date(2025,3,23), date(2025,4,1))
        res2.confirm_booking()
    except RoomNotAvailableError as e:
        print(f" Caught double booking: {e}")

    # 3. Test payment failure
    invoice = Invoice(1, res1)
    payment = Payment(1, invoice, "Cryptocurrency")  # Unsupported method

    try:
        payment.process_payment()
    except PaymentProcessingError as e:
        print(f" Caught payment error: {e}")

except Exception as e:
    print(f" Critical system failure: {e}")
finally:
    print("\n=== Error Handling Test Complete ===")

 Booking Failed: Room 101 is already booked
 Caught double booking: Room 101 is already booked
 Critical system failure: Payment() takes no arguments

=== Error Handling Test Complete ===
